In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import gmaps
import os
import time
import datetime

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
# Creates lists for data to be stored 
date = []
place = []
mag = []
cdi = []
mmi = []
coordinates = []
lat = []
long = []

# 1857 Fort Tejon Earthquake Data
def earthquake(e):
    params = {"starttime": "1800-01-01", "endtime": "1857-01-20",
              "minmagnitude": "4.0", "maxradiuskm":600, "latitude": 34.21, "longitude":-118.57}
    data = requests.get(e, params = params)
    data = json.loads(data.text)
    return data

e = r"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
a = earthquake(e)

for i in (a["features"]):
    print(i["properties"]["time"], i["properties"]["place"],
    i["properties"]["cdi"], i["properties"]["mag"],i["properties"]["mmi"],i["geometry"]["coordinates"][0],i["geometry"]["coordinates"][1])

-3564544020000 Near Palmdale, California None 6.3 None -118.04 34.52
-3565099200000 Southeast of Fort Tejon, California None 5.6 None -118.71 34.76
-3565150560000 The 1857 Fort Tejon Earthquake None 7.93 None -120.3 35.7
-3565155600000 West of Coalinga, California None 5.6 None -120.85 36.29
-3565159200000 West of Coalinga, California None 6.1 None -120.65 36.1
-3574686600000 Near Santa Ysabel, California None 5.5 None -116.7 33.1
-3593586900000 San Mateo County, California None 5.9 None -122.3 37.5
-3597371100000 Offshore San Mateo County, California None 5.7 None -122.5 37.3
-3608456400000 North of San Francisco, California None 5.5 None -122.5 38.1
-3612541500000 Near San Gabriel, California None 6 None -118.1 34.1
-3647594100000 Near Santa Barbara, California None 6 None -119.7 34.45
-3671049600000 Southeast of Mission Soledad, California None 6.3 None -120.8 36.25
-3689377200000 Near San Simeon, California None 5.5 None -121.1 35.6
-3694910400000 Baja California, Mexico None 6.5 N

In [3]:
for i in (a["features"]):
    date.append(i["properties"]["time"])
    place.append(i["properties"]["place"])
    cdi.append(i["properties"]["cdi"])
    mag.append(i["properties"]["mag"])
    mmi.append(i["properties"]["mmi"])
    lat.append(i["geometry"]["coordinates"][0])
    long.append(i["geometry"]["coordinates"][1])
    
# create a dictionary for establishing dataframe
fort_tejon = {
    "City":place,
    "Epoch Time":date,
    "Magnitude": mag,
    "MMI": mmi,
    "CDI":cdi,
    "Latitude":lat,
    "Longitude":long
}

fort_tejon_df = pd.DataFrame(fort_tejon)
fort_tejon_df.head()

,City,Epoch Time,Magnitude,MMI,CDI,Latitude,Longitude
0,"Near Palmdale, California",-3564544020000,6.30,None,None,-118.04,34.52
1,"Southeast of Fort Tejon, California",-3565099200000,5.60,None,None,-118.71,34.76
2,The 1857 Fort Tejon Earthquake,-3565150560000,7.93,None,None,-120.30,35.70
3,"West of Coalinga, California",-3565155600000,5.60,None,None,-120.85,36.29
4,"West of Coalinga, California",-3565159200000,6.10,None,None,-120.65,36.10


In [4]:
epoch_list = fort_tejon_df['Epoch Time'].tolist()
converted_list = []

for i in epoch_list:
    converted = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(i/1000.0))
    converted_list.append(converted)

In [5]:
fort_tejon_df['Date & Time'] = converted_list
fort_tejon_df.head()

,City,Epoch Time,Magnitude,MMI,CDI,Latitude,Longitude,Date & Time
0,"Near Palmdale, California",-3564544020000,6.30,None,None,-118.04,34.52,1857-01-16 16:53:00
1,"Southeast of Fort Tejon, California",-3565099200000,5.60,None,None,-118.71,34.76,1857-01-10 06:40:00
2,The 1857 Fort Tejon Earthquake,-3565150560000,7.93,None,None,-120.30,35.70,1857-01-09 16:24:00
3,"West of Coalinga, California",-3565155600000,5.60,None,None,-120.85,36.29,1857-01-09 15:00:00
4,"West of Coalinga, California",-3565159200000,6.10,None,None,-120.65,36.10,1857-01-09 14:00:00


In [6]:
coordinates = fort_tejon_df[['Longitude', 'Latitude']]

In [7]:
# Plotting map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Adding marker layer
## TODO: Make markers interactive
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…